In [23]:
# imports

import requests
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [40]:
def get_data(url):
    """
    Get data from a given url
    """
    return BeautifulSoup(requests.get(url).text, 'html.parser')

request_url = 'https://www.serebii.net/brilliantdiamondshiningpearl/poffin.shtml'

def get_map_from_stats_row(stats_row):
    map_from_stats_row = {}

    left = 0
    i = 0
    while i < len(stats_row):
        char = stats_row[i]

        if char.isnumeric():
            right = i
            val_for_map = ''

            while i < len(stats_row) and stats_row[i].isnumeric():
                val_for_map += stats_row[i]
                i += 1

            map_from_stats_row[stats_row[left:right]] = int(val_for_map)
            left = i
            continue
        i += 1
    
    return map_from_stats_row

def get_picture_using_name(name):
    berry_name = name.split()[0].lower()
    return f'/berries/{berry_name}.png'

def get_df_from_html(html):
    df_from_html = pd.read_html(str(html), header=0)[0]

    df_from_html['Spicy'] = [-1 for _ in range(len(df_from_html))]
    df_from_html['Dry'] = [-1 for _ in range(len(df_from_html))]
    df_from_html['Sweet'] = [-1 for _ in range(len(df_from_html))]
    df_from_html['Bitter'] = [-1 for _ in range(len(df_from_html))]
    df_from_html['Sour'] = [-1 for _ in range(len(df_from_html))]
    df_from_html['Smoothness'] = [-1 for _ in range(len(df_from_html))]

    for idx in df_from_html.index:
        df_from_html.loc[idx, 'Picture'] = get_picture_using_name(df_from_html.loc[idx, 'Name'])
        stats_row = df_from_html.loc[idx, 'Stats']
        map_from_stats_row = get_map_from_stats_row(stats_row)
        
        for key in map_from_stats_row.keys():
            df_from_html.loc[idx, key] = map_from_stats_row[key]
    
    df_from_html.drop(columns=['Stats'], inplace=True)
    
    return df_from_html

def export_df_to_json(df, filename):
    df.to_json(f'../public/{filename}.json', orient='records')
        

In [38]:
text = get_data(request_url)

In [41]:
all_rows = text.find_all('table')

df_from_html = get_df_from_html(all_rows[2])
export_df_to_json(df_from_html, 'berries-info')